In [68]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import math

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: Qt5Agg


In [2]:
RemoveSecondLook = False
StartFromBeginning = False
Beginning = False
End = False

In [3]:
TimeDwellOrig = 800
TimeFixation = 300

In [4]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [18]:
# Create list of list (made of epochs) composed of times when user looked at scratchPad

def UserLookedAtScratchPad(keysLookedAt):
    epoch = list()   
    epochList = list()

    # TODO: reduce the list to only the first and the last elements of the epoch
    
    nRow = -1
    firstScratchPad = 0
    flagStart = 0
    
    
    # Find unique looks at scratchpad, by checking if progress pct is greater than previous one
    for row in keysLookedAt:
        nRow = nRow + 1
        
        if row[1] == 'ScratchPad':
            
            # for consecutive scratchpad looks
            if keysLookedAt[nRow-1][1] == 'ScratchPad':
                if float(row[2]) == 0:
                    progressScratchPad = float(keysLookedAt[nRow-1][2])
                else:
                    progressScratchPad = float(row[2])
    
            # if first time of the look at scratchpad
            else:
                # if not the very first look (means the variable for progress exists)
                if firstScratchPad > 0: 
                    # if the current progress value is less than the previous one, means it is a new look 
                    if progressScratchPad > float(row[2]):
                        # if progress is not 0, then update the progress --> sometimes, it is 0 at the end of the look
                        if float(row[2]) != 0:
                            progressScratchPad = float(row[2])
                            epochList.append(row)
                            
                # if the very first look (means the variable for progress does not exist)    
                else:
                    progressScratchPad = float(row[2])
                    firstScratchPad = 1
                    epochList.append(row)
    
    #print(epochList)

    return epochList

In [6]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [7]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
    
    return nearestTP, nearestTPind

In [52]:
def UserLooksLocationAfterScratchPad(userKeys, scratchPadEpochs):
    
    scratchPad5sTimeEpoch = list()
    scratchPad5sTimeList = list()
    
    
    # create time list of time in userKeys
    userKeysTimeString = [key[0] for key in userKeys]
    userKeysTime = timeConversion(userKeysTimeString)
    
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in userKeys]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
            
    
    # scratchPad time
    scratchPadTimeString = [key[0] for key in scratchPadEpochs]
    scratchPadTime = timeConversion(scratchPadTimeString)
    
    
    if StartFromBeginning:
        
        if Beginning:
        
            # beginning
            timeStart = userKeysTime[0]
            timeStartInd = 0 
            timeAfter5s = timeStart + datetime.timedelta(seconds=10) 
            timeEnd = nearestTimePoint(userKeysTime, timeAfter5s)
            scratchPad5sTimeEpoch.append([timeStart, timeStartInd, timeEnd[0], timeEnd[1]])
        
        if End:
            # end
            timeEndInd = indBeforeQuit
            timeEnd = userKeysTime[indBeforeQuit]
            timeStartBefore5s = timeEnd - datetime.timedelta(seconds=10)
            timeStart = nearestTimePoint(userKeysTime, timeStartBefore5s)
            scratchPad5sTimeEpoch.append([timeStart[0], timeStart[1], timeEnd, timeEndInd])
        
    else:
        # create list of time of 5s after every look at scratchPad
        for key in scratchPadTime:
        
            timeStart = key
            timeStartInd = userKeysTime.index(timeStart)
            timeAfter5s = timeStart + datetime.timedelta(seconds=5) 
        
            #print(timeStart, timeAfter5s)
        
            timeEnd = nearestTimePoint(userKeysTime, timeAfter5s)
            scratchPad5sTimeEpoch.append([timeStart, timeStartInd, timeEnd[0], timeEnd[1]])
        
            #print(timeStart, timeEnd[0])
    
    
    #print(len(scratchPad5sTimeEpoch))
    
    #if len(scratchPad5sTimeEpoch) < 2:
    #    scratchPad5sTimeList = userKeys[scratchPad5sTimeEpoch[0][1] : scratchPad5sTimeEpoch[0][3]]
    #else:
    #    # create the 5s list
    #    for keyInd in range(0, len(scratchPad5sTimeEpoch)):
    #        scratchPad5sTimeList.append(userKeys[scratchPad5sTimeEpoch[keyInd][1] : scratchPad5sTimeEpoch[keyInd][3]])
        
    scratchPad5sTimeList = [userKeys[scratchPad5sTimeEpoch[keyInd][1]:scratchPad5sTimeEpoch[keyInd][3]] for keyInd in range(0, len(scratchPadEpochs))]
    
    
    scratchPad5sReducedList = list()
    
    if RemoveSecondLook:
        print('remove second look')
        # remove a second look of scratchPad continued in the same look
        # start checking from the back of the list, and if a key other than scratchPad or suggestion exists, then list is 
        # stopped there
        listInd = -1
        userKeysRemove = list()
        for userLooksList in scratchPad5sTimeList:
            
            
            # list to store if second looks present in the same list
            indToRemove = 0
        
            listInd = listInd + 1
            # for every look, ensure that the first key reported is the scratchPad
            if userLooksList[0][1] != 'ScratchPad':
                #print(scratchPad5sTimeList[listInd][0])
                scratchPad5sTimeList[listInd].remove(scratchPad5sTimeList[listInd][0])
        
            # create list of index of looks at scratchPad
            scratchPadInd = [rowInd for rowInd in range(0, len(userLooksList)) if userLooksList[rowInd][1] in keyLocation['scratchPadAndSuggestions'] or userLooksList[rowInd][1] in keyLocation['phraseText']]
               
            # create a list with difference in index of looks at scratchPad
            scratchPadIndDiffArray = np.diff(np.asarray(scratchPadInd), 1)
            scratchPadIndDiff = scratchPadIndDiffArray.tolist()
        
            # create list of keys of the scratchPads
            scratchPadKeys = [row for row in userLooksList if row[1] in keyLocation['scratchPadAndSuggestions'] or row[1] in keyLocation['phraseText']]

            # if another look started, remove the rows of that second look
            if len(scratchPadInd) > 1:
                # continue only if scratchPadInd has more than 1 element
            
                for indofInd in range(0, len(scratchPadIndDiff)):
                    ind = scratchPadIndDiff[indofInd]
                    if ind > 2: # to avoid inadvertant saccades to neighbouring keys
                    
                        # store the (index-1) from where it should be removed, in short, till where to save the list
                        indToRemove = indofInd + ind
                    
                        # inadvertant looks to scratchPad need to be removed, i the look had a progress of less than 300ms
                    
                        # check if the indToRemove is of scratchPad, and that the progress is more than 0, otherwise, 
                        # set indToRemove to 0
                        if scratchPad5sTimeList[listInd][indToRemove][1] == 'ScratchPad':
                            if float(scratchPad5sTimeList[listInd][indToRemove][2]) != 0:
                                # if progress is more than 0
                                # check if the previous scratchPad look was continued
                            
                                if float(scratchPad5sTimeList[listInd][scratchPadInd.index(indofInd)][2]) >= float(scratchPad5sTimeList[listInd][indToRemove][2]):
                                    #print('still the same look')
                                    indToRemove = 0
                                else:
                                    #print(len(userLooksList), indToRemove)
                                    break
                                    
                            else:
                                indToRemove = 0
    
                        else:
                            indToRemove = 0
                            continue
                    
                    
                    
            if indToRemove > 0:
                scratchPad5sReducedList.append(scratchPad5sTimeList[listInd][0:indToRemove])
                print('REMOVE', len(scratchPad5sTimeList[listInd][0:indToRemove]))
            else:
                # if indToRemove is still 0, add the whole list 
                scratchPad5sReducedList.append(scratchPad5sTimeList[listInd])
                print('INTACT', len(scratchPad5sTimeList[listInd]))
    else:
        print('keep 5s')
        # keep complete 5 s
        #print(len(scratchPad5sTimeList))
        scratchPad5sReducedList = scratchPad5sTimeList
        
    
        
    return scratchPad5sReducedList

In [53]:
keyLocation = dict()

keyLocation['scratchPadAndSuggestions'] = ['ScratchPad', 'NextSuggestions', 'Suggestion1', 'Suggestion2', 'Suggestion3', 'Suggestion4', 
                             'Suggestion5']
keyLocation['phraseText'] = ['PhraseTextBlock']
keyLocation['sleepKey'] = ['Sleep']
keyLocation['nextPhraseOrQuit'] = ['NextPhrase', 'Quit', 'YesQuestionResult']
keyLocation['characterKey'] = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', 'å', 'a', 's', 'd', 'f', 'g', 'h', 'j', 'k',
                               'l', 'æ', 'ø', 'z', 'x', 'c', 'v', 'b', 'n', 'm', '.', ',', 'SpaceBar', 'LeftShift']
keyLocation['dwellTime'] = ['IncreaseDwellTime', 'DecreaseDwellTime']

In [54]:
def AggregateUserLookLocationOneLook(scratchPad5sList, categoryList, keyLocation, subjName):
    
    typingMechanism = subjName[-2:]
    # go through the list and divide into categories of scratchPad + suggestions, character keys, sleep, phraseText or Quit 
    # and others
    
    if typingMechanism == 'DT':
        keyInd = - 1
        for key in scratchPad5sList:
            keyInd = keyInd + 1
        
            if len(categoryList['DT']) > keyInd:
            # this list exists and the values need only be modified
                if key[1] in keyLocation['scratchPadAndSuggestions']:
                    categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                    #print(key)
                elif key[1] in keyLocation['phraseText']:
                    categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                elif key[1] in keyLocation['sleepKey']:
                    categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                elif key[1] in keyLocation['nextPhraseOrQuit']:
                    categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                elif key[1] in keyLocation['characterKey']:
                    categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
                else: 
                    # others
                    #print(key)
                    categoryList['DT'][keyInd][5] = categoryList['DT'][keyInd][5] + 1
              
            else:
                # add a list of 6 elements to the list element
                categoryList['DT'].append([0, 0, 0, 0, 0, 0])
                # list does not exist and values need be appended
                if key[1] in keyLocation['scratchPadAndSuggestions']:
                    categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                elif key[1] in keyLocation['phraseText']:
                    categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                elif key[1] in keyLocation['sleepKey']:
                    categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                elif key[1] in keyLocation['nextPhraseOrQuit']:
                    categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                elif key[1] in keyLocation['characterKey']:
                    categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
                else: 
                    # others
                    #print(key)
                    categoryList['DT'][keyInd][5] = categoryList['DT'][keyInd][5] + 1
        
    else:
        
        keyInd = -1
        for key in scratchPad5sList:
            keyInd = keyInd + 1
            
            if len(categoryList['MS']) > keyInd:
                # this list exists and the values need only be modified
                if key[1] in keyLocation['scratchPadAndSuggestions']:
                    categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                    #print(key)
                elif key[1] in keyLocation['phraseText']:
                    categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                elif key[1] in keyLocation['sleepKey']:
                    categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                elif key[1] in keyLocation['nextPhraseOrQuit']:
                    categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
                elif key[1] in keyLocation['characterKey']:
                    categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
                else: 
                    # others
                    #print(key)
                    categoryList['MS'][keyInd][5] = categoryList['MS'][keyInd][5] + 1
                    
              
            else:
                # add a list of 6 elements to the list element
                categoryList['MS'].append([0, 0, 0, 0, 0, 0])
                # list does not exist and values need be appended
                    
                if key[1] in keyLocation['scratchPadAndSuggestions']:
                    categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                elif key[1] in keyLocation['phraseText']:
                    categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                elif key[1] in keyLocation['sleepKey']:
                    categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                elif key[1] in keyLocation['nextPhraseOrQuit']:
                    categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
            
                elif key[1] in keyLocation['characterKey']:
                    categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
                else: 
                    # others
                    #print(key)
                    categoryList['MS'][keyInd][5] = categoryList['MS'][keyInd][5] + 1
     
        
    return categoryList

In [55]:
def AggregateUserLookLocation(scratchPad5sList, categoryList, keyLocation, subjName):
    
    typingMechanism = subjName[-2:]
    # go through the list and divide into categories of scratchPad + suggestions, character keys, sleep, phraseText or Quit 
    # and others
    
    if typingMechanism == 'DT':
        for list5s in scratchPad5sList:
            keyInd = -1
            
            for key in list5s:
                keyInd = keyInd + 1
            
                if len(categoryList['DT']) > keyInd:
                    # this list exists and the values need only be modified
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                        #print(key)
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                    elif key[1] in keyLocation['sleepKey']:
                        categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuit']:
                        categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['DT'][keyInd][5] = categoryList['DT'][keyInd][5] + 1
              
                else:
                    # add a list of 6 elements to the list element
                    categoryList['DT'].append([0, 0, 0, 0, 0, 0])
                    # list does not exist and values need be appended
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['DT'][keyInd][0] = categoryList['DT'][keyInd][0] + 1
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['DT'][keyInd][1] = categoryList['DT'][keyInd][1] + 1
                    elif key[1] in keyLocation['sleepKey']:
                        categoryList['DT'][keyInd][2] = categoryList['DT'][keyInd][2] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuit']:
                        categoryList['DT'][keyInd][3] = categoryList['DT'][keyInd][3] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['DT'][keyInd][4] = categoryList['DT'][keyInd][4] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['DT'][keyInd][5] = categoryList['DT'][keyInd][5] + 1
        
    else:
        
        for list5s in scratchPad5sList:
            keyInd = -1

            for key in list5s:
                keyInd = keyInd + 1
            
                if len(categoryList['MS']) > keyInd:
                    # this list exists and the values need only be modified
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                        #print(key)
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                    elif key[1] in keyLocation['sleepKey']:
                        categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuit']:
                        categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['MS'][keyInd][5] = categoryList['MS'][keyInd][5] + 1
                    
              
                else:
                    # add a list of 6 elements to the list element
                    categoryList['MS'].append([0, 0, 0, 0, 0, 0])
                    # list does not exist and values need be appended
                    
                    if key[1] in keyLocation['scratchPadAndSuggestions']:
                        categoryList['MS'][keyInd][0] = categoryList['MS'][keyInd][0] + 1
                    elif key[1] in keyLocation['phraseText']:
                        categoryList['MS'][keyInd][1] = categoryList['MS'][keyInd][1] + 1
                    elif key[1] in keyLocation['sleepKey']:
                        categoryList['MS'][keyInd][2] = categoryList['MS'][keyInd][2] + 1
                    elif key[1] in keyLocation['nextPhraseOrQuit']:
                        categoryList['MS'][keyInd][3] = categoryList['MS'][keyInd][3] + 1
                    elif key[1] in keyLocation['characterKey']:
                        categoryList['MS'][keyInd][4] = categoryList['MS'][keyInd][4] + 1
                    else: 
                        # others
                        #print(key)
                        categoryList['MS'][keyInd][5] = categoryList['MS'][keyInd][5] + 1
     
        
    return categoryList

In [59]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
categoryList = dict()
categoryList['DT'] = list()
categoryList['MS'] = list()

firstSubj = 0

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            else:
                continue
            
            if userKeys is None:
                continue
            else:
                
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)
                
                # count % of people looking at what
                
                scratchPadEpochs = UserLookedAtScratchPad(userKeysWithTime)
                
                #print(scratchPadEpochs)
                
                scratchPad5sList =  UserLooksLocationAfterScratchPad(userKeysWithTime, scratchPadEpochs)
                
                #print(scratchPad5sList)
                if StartFromBeginning:
                    categoryList = AggregateUserLookLocationOneLook(scratchPad5sList, categoryList, keyLocation, subjName)
                else:
                    categoryList = AggregateUserLookLocation(scratchPad5sList, categoryList, keyLocation, subjName)
                
                

akt_MS
keep 5s
jl_DT
keep 5s
KEA_MS
keep 5s
lone_DT
keep 5s
mcc_MS
keep 5s
MK_DT
keep 5s
ok_MS
keep 5s
pt_DT
keep 5s
sc_MS
keep 5s
sh_MS
keep 5s
ae_DT
keep 5s
hc_MS
keep 5s
ib_MS
keep 5s
km_DT
keep 5s
ma_DT
keep 5s
pgba_DT
keep 5s
smn_DT
keep 5s
eo_DT
keep 5s
jek_MS
keep 5s
jg_DT
keep 5s
lg_MS
keep 5s
lr_MS
keep 5s
mm_MS
keep 5s
snk_DT
keep 5s


In [57]:
# make both lists equal

print(len(categoryList['DT']), len(categoryList['MS']))

if (len(categoryList['DT']) - len(categoryList['MS'])) > 0:
    
    for i in range(0, len(categoryList['DT']) - len(categoryList['MS'])):
        categoryList['MS'].append([0]*6)

330 276


In [94]:
# plot the category list

fig = plt.figure()

# normalise the category list
categoryListDT = categoryList['DT']
rowInd = -1
for row in categoryListDT:
    rowInd = rowInd + 1
    sumOfRow = sum(row)
    categoryInd = -1
    for category in row:
        categoryInd = categoryInd + 1
        categoryListDT[rowInd][categoryInd] = categoryListDT[rowInd][categoryInd]/sumOfRow
        
# normalise the category list
categoryListMS = categoryList['MS']
rowInd = -1
for row in categoryListMS:
    rowInd = rowInd + 1
    sumOfRow = sum(row)
    if sumOfRow == 0:
        sumOfRow = 1
    categoryInd = -1
    for category in row:
        categoryInd = categoryInd + 1
        categoryListMS[rowInd][categoryInd] = categoryListMS[rowInd][categoryInd]/sumOfRow

# convert category list to individual lists
scratchPadAndSuggestionsListDT = [key[0] for key in categoryListDT]
characterListDT = [key[4] for key in categoryListDT]
phraseTextListDT = [key[1] for key in categoryListDT]
#sleepListDT = [key[2] for key in categoryListDT]
nextPhraseOrQuitOrSleepListDT = [key[2] + key[3] for key in categoryListDT]
dwellTimeListDT = [key[5] for key in categoryListDT]

# convert category list to individual lists
scratchPadAndSuggestionsListMS = [key[0] for key in categoryListMS]
characterListMS = [key[4] for key in categoryListMS]
phraseTextListMS = [key[1] for key in categoryListMS]
#sleepListMS = [key[2] for key in categoryListMS]
nextPhraseOrQuitOrSleepListMS = [key[2]+key[3] for key in categoryListMS]
dwellTimeListMS = [key[5] for key in categoryListMS]

xaxisDT = np.arange(0, len(categoryList['DT']), 1)
#xaxis_ticksDT = np.arange(0, 5.3, 0.015)

bottomForNextPhraseDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] for i in range(0, len(characterListDT))]
bottomForPhraseTextDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] + nextPhraseOrQuitOrSleepListDT[i] for i in range(0, len(characterListDT))]
bottomForDwellTimeDT = [scratchPadAndSuggestionsListDT[i] + characterListDT[i] + nextPhraseOrQuitOrSleepListDT[i] + phraseTextListDT[i] for i in range(0, len(characterListDT))]

ax1 = fig.add_subplot(2,1,1)
ax1.bar(xaxisDT, scratchPadAndSuggestionsListDT)
ax1.bar(xaxisDT, characterListDT, bottom = scratchPadAndSuggestionsListDT)
#ax1.bar(xaxisDT, sleepListDT, label = 'sleep key')
ax1.bar(xaxisDT, nextPhraseOrQuitOrSleepListDT, bottom = bottomForNextPhraseDT)
ax1.bar(xaxisDT, phraseTextListDT, bottom = bottomForPhraseTextDT)
#ax1.bar(xaxisDT, dwellTimeListDT, bottom = bottomForDwellTimeDT, label = 'dwell time changed')
#ax1.legend( loc = 'upper right')


ax1.set_xlim([0, len(categoryList['DT'])])
#ax1.set_xticklabels([round(xaxis_ticksDT[0], 2), round(xaxis_ticksDT[50], 2) , round(xaxis_ticksDT[100], 2) , round(xaxis_ticksDT[150], 2) , round(xaxis_ticksDT[200], 2) , round(xaxis_ticksDT[250], 2), round(xaxis_ticksDT[300], 2), round(xaxis_ticksDT[-1], 2)])
ax1.set_title('Dwell time')
ax1.set_xlabel('Samples')
ax1.set_ylabel('Cumulative Frequency')

xaxisMS = np.arange(0, len(categoryList['MS']), 1)
xaxis_ticksMS = np.arange(0, 5.15, 0.015)

ax2 = fig.add_subplot(2,1,2)

bottomForNextPhraseMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] for i in range(0, len(characterListMS))]
bottomForPhraseTextMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] + nextPhraseOrQuitOrSleepListMS[i] for i in range(0, len(characterListMS))]
bottomForDwellTimeMS = [scratchPadAndSuggestionsListMS[i] + characterListMS[i] + nextPhraseOrQuitOrSleepListMS[i] + phraseTextListMS[i] for i in range(0, len(characterListMS))]


ax2.bar(xaxisMS, scratchPadAndSuggestionsListMS, label = 'scratchpad and suggestions')
ax2.bar(xaxisMS, characterListMS, bottom = scratchPadAndSuggestionsListMS, label = 'character keys')
#ax2.bar(xaxisMS, sleepListMS, label = 'sleep key')
ax2.bar(xaxisMS, nextPhraseOrQuitOrSleepListMS, bottom = bottomForNextPhraseMS, label = 'next phrase or quit or sleep')
ax2.bar(xaxisMS, phraseTextListMS, bottom = bottomForPhraseTextMS, label = 'phrase text')
#ax2.bar(xaxisMS, dwellTimeListMS, bottom = bottomForDwellTimeMS, label = 'dwell time changed')
fig.legend()

ax2.set_xlim([0, len(categoryList['DT'])])
#ax2.set_xticklabels([round(xaxis_ticksDT[0], 2), round(xaxis_ticksDT[50], 2) , round(xaxis_ticksDT[100], 2) , round(xaxis_ticksDT[150], 2) , round(xaxis_ticksDT[200], 2) , round(xaxis_ticksDT[250], 2), round(xaxis_ticksDT[300], 2), round(xaxis_ticksDT[-1], 2)])
ax2.set_title('Multi-key Selection')
ax2.set_xlabel('Samples')
ax2.set_ylabel('Cumulative Frequency')
plt.suptitle('5s window after looking at scratchpad', fontsize=16)

Text(0.5,0.98,'5s window after looking at scratchpad')

In [ ]:
sum([0.06451612903225806, 0.03225806451612903, 0.03225806451612903, 0.0, 0.8709677419354839, 0.0])

In [ ]:
# average time difference between consecutive userKeys

timeDiff = list()

for i in range(0, len(userKeys)):
    if i == 0:
        continue
    
    time1, t1, t2 = userKeys[i-1][0].partition('+')
    previousTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")

    time2, t1, t2 = userKeys[i][0].partition('+')
    currentTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    timeDiff.append((currentTime - previousTime).total_seconds())
    
print('mean: ', np.mean(timeDiff))
print('median: ', np.median(timeDiff))
print('min: ', np.min(timeDiff))
print('max: ', np.max(timeDiff))

#### If median of time difference is taken as step to plot, it is the same as the sampling frequency